In [1]:
from hhsolver import *
import aspose.words as aw
from PIL import Image
from hhsampling import *
from utils import *

ModuleNotFoundError: No module named 'pandas'

In [1]:
%load_ext autoreload


In [3]:
%autoreload 2

In [6]:
file_path = '../data/Candidatos20230803_n.txt'
data = collect_data(file_path=file_path).to_numpy()

lambda_n = 0#, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

nalloc = 10

sample_size = [60]#[60, 120, 180, 240] 

fileNames = []

time_limit_list = [240]#[5,10,20,30,60,120,240]

# hhsamplingGA require transformed data
x = data @ inversecholcov(data)
solver = HHSolverPSO

In [7]:
#Teste com lambda 0
fig, axs = plt.subplots(len(sample_size), 1, figsize=(8, 10), sharex=True)
min = []
for index, sample in enumerate(sample_size):
        # pre-generate noises used in each batch
        ma = 2
        noise = []
        populations = []
        for i in range(nalloc):
            z = generate_noise(x.shape[0], ma)
            # hhsamplingGA require transformed noise
            zt = z @ inversecholcov(z)
            noise.append(zt)
            populations.append(generate_pop(50, sample, x.shape[0]))

        stats_aux = {"distq95": [], "fkappa": []}

        for time_limit in time_limit_list:
            res = hhsampling(x, lambda_n, nalloc, sample, noise, populations, solver, time_limit)
            for k in stats_aux:
                stats_aux[k].append(res[k])
            
            dist = list(np.array(res["dist"]))
            plt.subplot(len(sample_size), 1, index+1)
            
            plt.hist(x = dist)
            #min.append(min(dist))
            plt.title(f'Sample Size: {sample}') 
       
plt.tight_layout()
plt.show()
